In [ ]:
#-----------------------------------
# Import packages
#-----------------------------------
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
import scipy.optimize as opt
import pylab as pl
%matplotlib inline

# Downloading file
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
#Reading file
df = pd.read_csv('loan_train.csv')
df.head()
df.shape

# Convert to date time object
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()
#---------------------------------------

# DATA VISUALIZATION AND PRE PROCESSING

#---------------------------------------

df['loan_status'].value_counts()

import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# PRE PROCESSING 

df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

df.groupby(['education'])['loan_status'].value_counts(normalize=True)

df[['Principal','terms','age','Gender','education']].head()

# Feature Extraction

Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()
X = Feature
X[0:5]
y = df['loan_status'].values
y[0:5]

# Normalization

X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

#-----------------------------------------------------

## #Train Test Split

#-----------------------------------------------------



from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=4)
print('Train set in TTS:', X_train.shape, y_train.shape)
print('Test set in TTS:', X_test.shape, y_test.shape)

#-----------------------------------------------------

## MODELING

#-----------------------------------------------------


#KNN Classifier Model

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
k=4
neigh=KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
yKNNhat=neigh.predict(X_test)
Trainset_accuracy_initial=metrics.accuracy_score(y_train, neigh.predict(X_train))
print('Trainset accuracy initial', Trainset_accuracy_initial)
Testset_accuracy_initialKNN=metrics.accuracy_score(y_test, yKNNhat)
print('Testset accuracy initial', Testset_accuracy_initial)
#Compare k value
Ks = 10
mean_accKNN = np.zeros((Ks-1))
std_accKNN = np.zeros((Ks-1))
ConfustionMxKNN = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yKNNhat=neigh.predict(X_test)
    mean_accKNNN[n-1] = metrics.accuracy_score(y_test, yKNNhat)

    
    std_accKNN[n-1] = np.std(yhat==y_test)/np.sqrt(yKNNhat.shape[0])

mean_accKNN
k_new = mean_accKNN.argmax()+1
print(metrics.confusion_matrix(y_test,yhat))

#Plot neighbor accuracy for different k
plt.plot(range(1,Ks),mean_accKNN,'g')
plt.fill_between(range(1,Ks),mean_accKNN - 1 * std_accKNN,mean_accKNN + 1 * std_accKNN, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neibors (K)')
plt.tight_layout()
plt.show()

#RESULT
print( "The best accuracy was ", mean_accKNN.max(), "with k=", mean_accKNN.argmax()+1)
